In [2]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from scipy.stats import norm
import pdb

# creating black scholes price and greeks
class BS():
    
    def CallPrice(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/(np.sqrt(T)*sigma)
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/(np.sqrt(T)*sigma)
        
        return S*norm.cdf(dp) - K*np.exp(-r*T)*norm.cdf(dm)
    
    def PutPrice(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return K*np.exp(-r*T)*norm.cdf(-dm) - S*norm.cdf(-dp)
    
    
    def CallDelta(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return norm.cdf(dp)
    
    def PutDelta(S, T, K, sigma, r):
        
        return BS.CallDelta(S, T, K, sigma, r)-1
    
    def CallGamma(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return norm.pdf(dp)/(S*sigma*np.sqrt(T))
    
    def PutGamma(S, T, K, sigma, r):
        
        return BS.CallGamma(S, T, K, sigma, r)
    
    def CallVega(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return norm.pdf(dp)*S*np.sqrt(T)
    
    def CallTheta(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return -S*norm.pdf(dp)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(dm)
    
    def PutTheta(S, T, K, sigma, r):
        
        dp = (np.log(S/K) + (r+0.5*sigma**2)*T)/np.sqrt(T)/sigma
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return -S*norm.pdf(dp)*sigma/(2*np.sqrt(T)) + r*K*np.exp(-r*T)*norm.cdf(-dm)
    
    def CallRho(S, T, K, sigma, r):
        
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return K*T*np.exp(-r*T)*norm.pdf(dm)
    
    def PutRho(S, T, K, sigma, r):
        
        dm = (np.log(S/K) + (r-0.5*sigma**2)*T)/np.sqrt(T)/sigma
        
        return -K*T*np.exp(-r*T)*norm.pdf(-dm)    

In [5]:
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default='notebook' # set = 'browser' when not in Jupyter notebook

import pdb

#from BS import BS

# solve the pde
class BS_PDE():
    
    def __init__(self, T, F, f_exact, S0 = 100, sigma=0.2, r = 0.02, NdT = 1_000):
        
        self.NdT = NdT
        self.T = T
        self.t = np.linspace(0,self.T, self.NdT+1)
        self.dt = self.t[1] - self.t[0]
        
        self.S0 = S0
        self.sigma = sigma
        self.r = r
        
        self.dx = np.sqrt(3*self.dt)*self.sigma
        eff_std = self.sigma*np.sqrt(self.T)
        n_std = 3
        self.Ndx = int(2*n_std*eff_std/self.dx) + 1
        self.x = np.linspace(-n_std*eff_std,
                             n_std*eff_std, 
                             self.Ndx)
        self.dx = self.x[1]-self.x[0]
        
        self.S = self.S0 * np.exp(self.x)
        
        self.F = F
        self.f_exact = f_exact
        
    def Solve(self):
        
        f = np.zeros((self.Ndx, self.NdT+1))
        
        
        f[:,-1] = self.F(self.S)
        
        for i in range(self.NdT-1, -1, -1):
            
            d_dx_f = (f[2:,i+1] - f[:-2,i+1])/(2*self.dx)
            d_dxdx_f = (f[2:,i+1] - 2*f[1:-1,i+1] + f[:-2,i+1])/self.dx**2
            
            f[1:-1,i] = f[1:-1,i+1] + self.dt*((self.r-0.5*self.sigma**2) * d_dx_f
                                               + 0.5*self.sigma**2 * d_dxdx_f 
                                               - self.r*f[1:-1:,i+1])
            
            f[0,i] = 2*f[1,i] - f[2,i]
            f[-1,i] = 2*f[-2,i] - f[-3,i]
            
        fig = go.Figure(data=[go.Surface(x=self.t, y=self.S, z=f)])

        fig.update_layout(title='***', #autosize=False, width=600, height=500,
                          margin=dict(l=65, r=50, b=65, t=90),
                          scene=dict(
                              xaxis=dict(title='t'),
                              yaxis=dict(title='S', range=(self.S0*0.6,self.S0*1.4)),
                              zaxis=dict(title='f(t,S)'),))
        idx = -1
        for i in range(5):
            
            # fig.add_trace(go.Scatter3d(x=self.t[idx]*np.ones(self.t.shape), y=S, z=f[:,idx],mode='lines'))
            fig.add_trace(go.Scatter3d(x=self.t[idx]*np.ones(self.t.shape), 
                                       y=self.S[::5], 
                                       z=self.f_exact(self.S[::5], self.T - self.t[idx]),
                                       showlegend=False))
            
            idx -= int(self.NdT/5)
        
        fig.show()
        
        return f

In [66]:
# get s
def get_dS(dt, r, sigma, S_last):
    z = np.random.normal(0, dt, 1)[0]
    return r * S_last * dt + sigma * S_last * z

# get m
def get_M(M_last, r, dt, delta, delta_last, S, phi_equity):
    return M_last * np.exp(r * dt) - (delta - delta_last) * S - phi_equity * np.abs(delta - delta_last)

In [67]:
# Set up
S0 = 100
sigma = 0.2
r = 0.02

K = 100
T = int(365/4)
dt = 1
Ndt = int(T/dt)

P0 = BS.PutPrice(S0, T, K, sigma, r)
delta_0 = -BS.PutDelta(S0, T-0, K, sigma, r)

phi_equity = 0.005

In [69]:
np.random.seed(1004913329)
s_init = S0
M_init = P0 - delta_0 * S0 - phi_equity * np.abs(delta_0 * S0)

for day in np.linspace(0, T-1, Ndt):
    stock_price = s_init + get_dS(dt, r, sigma, s_init)
    delta = -BS.PutDelta(stock_price, T-day, K, sigma, r)
    delta_last = -BS.PutDelta(stock_price, T-day+1, K, sigma, r)
    money_account = get_M(M_init, r, dt, delta, delta_last, stock_price, phi_equity)
    M_init = money_account
    s_init = stock_price

Final_S = s_init + get_dS(dt, r, sigma, s_init)
Final_M = M_init + Final_S * delta * (1 - phi_equity)

print(Final_M)

24.80594462968063
